In [5]:
from pyscf import gto, scf
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from state_tensor import get_coeff_tensor

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)

ket = driver.get_random_mps(tag="GS", bond_dim=250, nroots=1)

csfs, coeffs = driver.get_csf_coefficients(ket, cutoff=0.05, iprint=1)
print(csfs)
print(coeffs)
mps = driver.get_mps_from_csf_coefficients(csfs, coeffs, tag="CMPS", dot=2)
impo = driver.get_identity_mpo()
print(driver.expectation(mps, impo, mps))
print(driver.expectation(mps, mpo, mps) / driver.expectation(mps, impo, mps))

energy = driver.dmrg(mpo, mps, n_sweeps=5, bond_dims=[500] * 5, noises=[1E-5],
    thrds=[1E-10] * 5, iprint=1)
print('Ground state energy = %20.15f' % energy)

dtrie finished 0.007800125000017033
Number of CSF =         90 (cutoff =      0.05)
Sum of weights of included CSF =    0.841336768706393

CSF          0 2+0022222-  =   -0.299698158843147
CSF          1 2+-+22-202  =    0.220902010285312
CSF          2 20+-22+22-  =    0.178947193536261
CSF          3 2+2-200222  =   -0.178146302563060
CSF          4 20+-2+2-22  =   -0.175710902697427
CSF          5 2+02+2-2-2  =    0.165700412167265
CSF          6 2220022202  =    0.158175953271960
CSF          7 2+2-022202  =    0.149605396591530
CSF          8 2+-+2+2-2-  =   -0.131870081584156
CSF          9 2+-+02-222  =   -0.123856411532300
CSF         10 200+22222-  =   -0.119848817785274
CSF         11 2+2-220220  =    0.116940060828886
CSF         12 222+2-+-20  =   -0.116265877640011
CSF         13 2+0-222202  =    0.114171222171083
CSF         14 2+2+2-+-2-  =   -0.112121308866003
CSF         15 2+-++222--  =    0.111635253009561
CSF         16 2+2-2++-2-  =   -0.109807088558088
CSF        

In [8]:
def coeff_to_MPS(coeff_tensor):
    tensor_shape = coeff_tensor.shape
    tensor_n = tensor_shape[0]
    tensor_rank = len(tensor_shape)
    n_dets = tensor_n ** tensor_rank

    if tensor_n != 4:
        raise ValueError("Each site must have 4 dimension")

    csf_matrix = np.empty((n_dets, tensor_rank), dtype=np.int32)
    sz_list = [0, 1, 2, 3]
    for k in range(n_dets):
        for i in range(tensor_rank):
            csf_matrix[k, i] = 0
        count = 0
        number = k
        while number > 0:
            remainder = number % 4
            csf_matrix[k, count] = sz_list[remainder]
            number //= 4
            count += 1

    dvals = np.reshape(coeff_tensor, (n_dets,))



    print(csf_matrix)
    print(dvals)
    n_sites = tensor_rank
    

    driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ,
                        n_threads=4)
    driver.initialize_system(n_sites, n_elec, spin, orb_sym=None)

    MPS = driver.get_mps_from_csf_coefficients(csf_matrix, dvals, "CMPS", 2)
    return MPS

In [7]:
site_dim = 4
n_sites = 4
coefficient_tensor = get_coeff_tensor(site_dim, n_sites)

MPS = coeff_to_MPS(coefficient_tensor)

print(MPS)

[[0 0 0 0]
 [1 0 0 0]
 [2 0 0 0]
 ...
 [1 3 3 3]
 [2 3 3 3]
 [3 3 3 3]]
[3.04513720e-02 5.28206913e-01 6.17091101e-01 3.16562209e-03
 5.88444245e-01 4.89192927e-01 5.08646334e-01 2.42353814e-01
 3.01441938e-01 8.11187950e-01 3.26084752e-01 4.39190906e-01
 5.89804709e-03 7.44811651e-01 8.92245773e-01 3.18770536e-01
 1.62051025e-01 5.69107487e-01 2.79669949e-01 5.96105610e-02
 7.50017936e-01 7.65451421e-01 7.00022023e-01 1.63846216e-03
 4.97467978e-01 1.81894853e-01 6.01260745e-01 8.98266949e-01
 2.76230735e-01 9.59669446e-01 3.23327693e-01 8.60673615e-01
 2.98062238e-01 4.87513433e-01 8.32557056e-01 2.82030543e-01
 3.86859571e-01 7.08759055e-01 8.84518955e-01 2.42532759e-01
 1.96219430e-01 3.72489740e-01 9.41169856e-01 5.19618217e-01
 7.55990910e-01 9.07343869e-01 5.88190727e-02 1.95115686e-01
 4.45051465e-01 6.23103017e-01 8.51222248e-01 9.66291571e-01
 7.91665502e-01 4.83269678e-01 3.05057486e-01 3.98575436e-01
 4.39207031e-01 5.48157991e-02 1.07847299e-01 9.67634530e-01
 7.44123644e-

IndexError: vector